In [3]:
!pip install git+https://github.com/openai/swarm.git
!pip install openai
!pip install firecrawl-py

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-we93a6jb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-we93a6jb
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 30.0 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=25999 sha256=965de2633a9d

In [4]:
import os
from firecrawl import FirecrawlApp
from swarm import Agent, Swarm
from openai import OpenAI
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openai import ChatCompletion

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "schema" in "FirecrawlApp.ExtractParams" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [46]:
# Configure API Keys

os.environ["OPENAI_API_KEY"]= ""

#Load dataset
df = pd.read_csv("https://raw.githubusercontent.com/jaydiaz2012/AI_First_Day_6_AI_Swarm/refs/heads/main/ai%20first%20sales%20data%20-%20sales_final(web).csv")

# Analyze dataset
def analyze_dataset():
    df['date'] = pd.to_datetime(df['date'])
    numerical_df = df.select_dtypes(include=np.number)

    summary = {
        "overview": df.describe(include="all").to_dict(),
        "null_values": df.isnull().sum().to_dict(),
        "correlations": numerical_df.corr().to_dict(),
    }
    return summary

def generate_web_insights():
    web_insights = {
        'source_pageviews': df.groupby('source')['pageviews'].sum(),
        'source_visits': df.groupby('source')['visits'].sum(),
        'source_transactions': df.groupby('source')['revenue'].sum(),
        'source_product_click': df.groupby('source')['ad spend'].sum()
    }
    return web_insights

def generate_web_visuals():
    visualizations = {}

    # Page Views Distribution
    plt.figure(figsize=(10, 6))
    sns.lineplot(df, x="source", y="pageviews")
    plt.title("Page Views Distribution")
    visualizations['page_views_distribution'] = plt.gcf()
    plt.close()

    # Visits Distribution
    plt.figure(figsize=(10, 6))
    sns.lineplot(df, x="source", y="visits")
    plt.title("Visits Distribution")
    visualizations['visits_distribution'] = plt.gcf()
    plt.close()

    # Transactions Trends
    plt.figure(figsize=(10, 6))
    sns.lineplot(df, x="source", y="transactions")
    plt.title("Transactions Trends")
    visualizations['transactions_trends'] = plt.gcf()
    plt.close()

    # Product Click Trends
    plt.figure(figsize=(10, 6))
    sns.lineplot(df, x="source", y="productClick")
    plt.title("Product Clicks Trends")
    visualizations['product_click_trends'] = plt.gcf()
    plt.close()

    return visualizations

web_analytics_agent = Agent(
    name="Web Analyst Agent",
    model="gpt-4o-mini",
    instructions="""You are a web analyst agent that cleans the dataset and report on the statistical summary of the dataset""",
    functions=[analyze_dataset],
)

web_insights_agent = Agent(
    name="Web Insights Agent",
    model="gpt-4o-mini",
    instructions="""You are a Web Insights Agent that provide insights about the data provided and summarize the insights from other agents, including actionable recommendations""",
    functions=[generate_web_insights]
)

web_insights_visuals_agent = Agent(
    name="Web_Visuals_Agent",
    model="gpt-4o-mini",
    instructions="""You are a data visualization expert that create visualizations on a given datase and gives summary of the data visuals""",
    fuctions=[generate_web_visuals]
)

if __name__ == "__main__":
    client = Swarm()

    web_analytics_response = client.run(
        agent=web_analytics_agent,
        messages=[{
            "role": "user",
            "content": "Please clean and analyze my dataset."
        }]
    )

    print("\nWeb Analytics Results:")
    print(web_analytics_response.messages[-1]["content"])

    web_insights_response = client.run(
        agent=web_insights_agent,
        messages=[{
            "role": "user",
            "content": "Please provide insights based from my dataset."
        }]
    )

    print("\nWeb Insights Results:")
    print(web_insights_response.messages[-1]["content"])

    web_insights_visuals_response = client.run(
        agent=web_insights_visuals_agent,
        messages=[{
            "role": "user",
            "content": "Please provide charts using Seaborn or Matplotlib using the data from the given dataset."
        }]
    )

    print("\nWeb Analytics Visuals Results:")
    print(web_insights_visuals_response.messages[-1]["content"])


Web Analytics Results:
Here is the statistical summary and analysis of your dataset:

### Overview of the Dataset

- **Total Records**: 52,721

#### Date
- **Mean**: May 19, 2020
- **Min**: January 1, 2020
- **Max**: September 30, 2020

#### Sources
- **Unique**: 22
- **Most Frequent Source**: Facebook (12,954 occurrences)

#### Medium
- **Unique**: 6
- **Most Frequent Medium**: CPC (16,833 occurrences)

#### Delivery Available
- **Unique**: 3
- **Most Frequent**: No Data (31,953 occurrences)

#### Device Type
- **Unique**: 3
- **Most Frequent Device**: Mobile (24,416 occurrences)

#### Promo Activated
- **Unique**: 2
- **Most Frequent**: No (27,548 occurrences)

#### Filter Used
- **Unique**: 2
- **Most Frequent**: No (37,290 occurrences)

### Numerical Metrics

#### Pageviews
- **Mean**: 583.76
- **Min**: 0
- **Max**: 34,832
- **Standard Deviation**: 1,452.00

#### Visits
- **Mean**: 127.12
- **Min**: 1
- **Max**: 6,975
- **Standard Deviation**: 384.87

#### Product Clicks
- **Mean*